In [ ]:
import os
from pathlib import Path
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import timm
from torchmetrics import Accuracy
import torch.optim
from torchvision.datasets import ImageFolder
from tqdm.auto import tqdm
from pathlib import Path
from PIL import Image

In [ ]:
class ResNet50Model(nn.Module):
    def __init__(self, num_classes=196):
        super().__init__()
        self.backbone = timm.create_model('resnet50', pretrained=False)
        in_features = self.backbone.get_classifier().in_features
        self.backbone.fc = nn.Linear(in_features, num_classes)  # 'fc' là classifier của ResNet

    def forward(self, x):
        return self.backbone(x)

In [ ]:
modelV0 = ResNet50Model(num_classes=196)
modelV0.load_state_dict(torch.load('/kaggle/input/resnet50/pytorch/default/1/resnet50.pth'))

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
accuracy_fn = Accuracy(task ='multiclass',num_classes=196).to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modelV0.parameters(), lr=0.001)

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0), ratio=(0.95, 1.05)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.05),
    transforms.RandomGrayscale(p=0.05),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 0.3)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.15), ratio=(0.5, 2.0))
])
test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
train_data = ImageFolder('/kaggle/input/stanford-car/dataset/train', transform=train_transforms)
test_data = ImageFolder('/kaggle/input/stanford-car/dataset/test', transform=test_transforms)
train_loader = DataLoader(train_data,batch_size=16,shuffle=True)
test_loader = DataLoader(test_data,batch_size=16,shuffle=False)

In [ ]:
epochs = 10
device = 'cuda' if torch.cuda.is_available() else 'cuda'
modelV0 = modelV0.to(device)
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []
for epoch in range(epochs):
    modelV0.train()
    train_loss,train_acc = 0,0
    for x,y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Training]"):
        x,y = x.to(device), y.to(device)
        y_pred = modelV0(x)
        loss = loss_fn(y_pred,y)
        acc = accuracy_fn(y,y_pred.argmax(dim=1))
        train_loss += loss.item()
        train_acc += acc.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader)
    train_acc /= len(train_loader)


    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)

    modelV0.eval()
    val_loss, val_acc = 0,0
    with torch.inference_mode():
        for x,y in tqdm(test_loader, desc=f"Epoch {epoch+1}/{epochs} [Validation]"):
            x,y = x.to(device), y.to(device)
            y_pred = modelV0(x)
            loss = loss_fn(y_pred,y)
            acc = accuracy_fn(y,y_pred.argmax(dim=1))
            val_loss += loss.item()
            val_acc += acc.item()

        val_loss /= len(test_loader)
        val_acc /= len(test_loader)

    val_loss_history.append(val_loss)
    val_acc_history.append(val_acc)

    print(f"Epoch : {epoch +1} | Train Loss :{train_loss:.3f}| Train Acc :{train_acc*100:.2f}% | Val Loss :{val_loss:.3f}| Val Acc :{val_acc*100:.2f}%")

In [ ]:
plt.figure(figsize=(12, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_loss_history, label='Train Loss')
plt.plot(range(1, epochs + 1), val_loss_history, label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), [acc * 100 for acc in train_acc_history], label='Train Accuracy') # Scale to percentage
plt.plot(range(1, epochs + 1), [acc * 100 for acc in val_acc_history], label='Validation Accuracy') # Scale to percentage
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
model_path = Path('/kaggle/working/')
model_path.mkdir(parents=True, exist_ok=True)

MODEL_SAVE_PATH = model_path / 'resnet50.pth'
torch.save(modelV0.state_dict(), MODEL_SAVE_PATH)

In [ ]:
class_name = train_data.classes
class_name

In [ ]:
modelV0=modelV0.to(device)
x = Image.open('/kaggle/input/bugatti-veyron-16-4-coupe-2009/2009_bugatti_veyron-164_coupe_base_fq_oem_3_1600x1067.jpg')
x = test_transforms(x).to(device)
x = x.unsqueeze(dim = 0)
class_name[modelV0(x).argmax(dim = 1).item()]

In [ ]:
from PIL import Image
modelV0=modelV0.to(device)
modelV0.eval()
with torch.inference_mode():
    x = Image.open('/kaggle/input/lamborghini-gallardo-lp-570-4-superleggera-2012/Used-2012-Lamborghini-Gallardo-LP-570-4-Superleggera-RARE-Grigio-Telesto-Paint--Front-Lift-Reverse-Cam-1697233534.jpg')
    x_transformed = test_transforms(x).to(device)
    x_input = x_transformed.unsqueeze(dim=0)

    outputs = modelV0(x_input) # Logits thô
    probabilities = torch.softmax(outputs, dim=1)

    predicted_idx = outputs.argmax(dim=1).item()
    predicted_label = class_name[predicted_idx]
    confidence = probabilities[0, predicted_idx].item()

    print(f"Predicted Index: {predicted_idx}")
    print(f"Predicted Label: {predicted_label}")
    print(f"Confidence: {confidence*100:.2f}%")

    # In ra top 5 dự đoán
    top5_prob, top5_indices = torch.topk(probabilities, 5)
    print("\nTop 5 Predictions:")
    for i in range(top5_prob.size(1)):
        class_label = class_name[top5_indices[0, i].item()]
        class_prob = top5_prob[0, i].item()
        print(f"{i+1}.{class_label}:{class_prob*100:.2f}%")

In [ ]:
from PIL import Image
import torch

modelV0.to(device)
modelV0.eval()

CONFIDENCE_THRESHOLD = 0.7

with torch.inference_mode():
    img_path = '/kaggle/input/xe-my-honda-vision/vision-khung-dap-han-laser-eSAF.jpg'
    try:
        x_pil = Image.open(img_path)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy tệp hình ảnh tại đường dẫn: {img_path}")
        exit()
    except Exception as e:
        print(f"Lỗi khi mở hình ảnh: {e}")
        exit()

    x_transformed = test_transforms(x_pil).to(device)
    x_input = x_transformed.unsqueeze(dim=0)

    outputs = modelV0(x_input)
    probabilities = torch.softmax(outputs, dim=1)

    confidence_value, predicted_idx_tensor = torch.max(probabilities, dim=1)
    predicted_idx = predicted_idx_tensor.item()
    confidence = confidence_value.item()

    if confidence >= CONFIDENCE_THRESHOLD:
        predicted_label = class_name[predicted_idx]
        print(f"Predicted Index: {predicted_idx}")
        print(f"Predicted Label: {predicted_label}")
        print(f"Confidence: {confidence*100:.2f}%")

        top5_prob, top5_indices = torch.topk(probabilities, 5)
        print("\nTop 5 Predictions:")
        for i in range(top5_prob.size(1)):
            class_label_top_k = class_name[top5_indices[0, i].item()]
            class_prob_top_k = top5_prob[0, i].item()
            print(f"  {i+1}. {class_label_top_k}: {class_prob_top_k*100:.2f}%")
    else:
        print("Predicted Label: Undefined (Confidence below threshold)")
        print(f"Highest Confidence Found: {confidence*100:.2f}% (for class: {class_name[predicted_idx]})")

        # Tùy chọn: Vẫn hiển thị top 5 ngay cả khi dưới ngưỡng để debug
        top5_prob, top5_indices = torch.topk(probabilities, 5)
        print("\nTop 5 Predictions (even if below threshold):")
        for i in range(top5_prob.size(1)):
            class_label_top_k = class_name[top5_indices[0, i].item()]
            class_prob_top_k = top5_prob[0, i].item()
            print(f"  {i+1}. {class_label_top_k}: {class_prob_top_k*100:.2f}%")